In [0]:
import pyspark.sql.functions as sf 
from pyspark.sql import Window 

In [0]:
df = spark.read.format("csv")\
               .option("header","true")\
                .option("samplingRatio","0.1")\
                .load("/FileStore/tables/invoices.csv")


Simple and Windowing Functions

In [0]:
date = sf.to_date("InvoiceDate","dd-MM-yyyy H.mm")
NumInvoice = sf.countDistinct("InvoiceNo").alias("NumInvoice")
TotalQuantity = sf.sum("Quantity").alias("TotalQuantity")
InvoiceValue = sf.expr("round(sum(Quantity*UnitPrice),2)").alias("InvoiceValue")
InvoiceValue1 = sf.round(sf.sum(df.Quantity *df.UnitPrice),2).alias("InvoiceValue1")

df4 = df.where(sf.year(date)==2010)\
        .groupBy("country",sf.weekofyear(date).alias("week"))\
        .agg(NumInvoice,TotalQuantity,InvoiceValue,InvoiceValue1)
    
df4.write.format("parquet")\
          .mode("overwrite")\
          .save("/FileStore/aggregateddata/")


In [0]:
df_parquet = spark.read.format("parquet").\
                        load("/FileStore/aggregateddata/")

Window Function

In [0]:

running_total_window = Window.partitionBy("country").\
                              orderBy("week").\
                              rowsBetween(Window.unboundedPreceding,Window.currentRow)

df_runningsum = df_parquet.withColumn("RunningSum",sf.sum("InvoiceValue").over(running_total_window))
display(df_runningsum)

Top N records

In [0]:
ranking_window = Window.partitionBy("country")\
                        .orderBy("InvoiceValue")\
                        .rowsBetween(Window.unboundedPreceding,Window.currentRow)

df_ranking = df4.withColumn("Rank",sf.dense_rank().over(ranking_window))\
                .where("Rank<=3")
display(df_ranking)